In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import requests
import pandas as pd
import re
from bs4 import BeautifulSoup
from datetime import datetime
import numpy as np
import pickle

# 카카오페이지에서 데이터 크롤링

In [ ]:
# 카카오페이지 로그인 후 쿠키 저장하는 파트

url = 'https://page.kakao.com'
options = Options()
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36")
data_list = []

driver = webdriver.Chrome()
driver.get(url)
time.sleep(3)

driver.execute_script(f"document.querySelector('button.pr-16pxr').click();")
time.sleep(3)

id_input = driver.find_element(By.CSS_SELECTOR, '#loginId--1')
id_input.send_keys('tkdehdjajsl@gmail.com')
pw_input = driver.find_element(By.CSS_SELECTOR, '#password--2')
pw_input.send_keys('gksruf9507')
time.sleep(5)

submit_btn = driver.find_element(By.CSS_SELECTOR, 'button.submit')
submit_btn.click()
time.sleep(5)

# 로그인 후 쿠키 저장
cookies = driver.get_cookies()
with open('kakao_cookies.pkl', 'wb') as f:
  pickle.dump(cookies, f)

driver.quit()

In [ ]:
# 스크롤 끝까지 내려서 링크 모두 가져오는 파트

url = 'https://page.kakao.com/menu/10011/screen/84'
options = Options()
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36")
data_list = []

driver = webdriver.Chrome(options=options)

data = []
try:
  driver.get(url)
  time.sleep(3)

  last_height = driver.execute_script("return document.body.scrollHeight")

  for _ in range(1): # 2067회 돌려야함
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)

  html = driver.page_source
  soup = BeautifulSoup(html, 'html.parser')

  atags = soup.select('div.flex.w-full.grow.flex-col.px-122pxr > div > div.flex.grow.flex-col > div.mb-4pxr.flex-col > div > div.flex.grow.flex-col > div > div > div > div > div > a')
  
  for atag in atags:
    img = atag.select_one('img')['src']
    title = atag.select_one('div.jsx-3256825605.font-small1.break-all.text-el-60.line-clamp-2').text
    data.append({'img': img, 'title': title})
    
except:
  print('Error occurred')

df = pd.DataFrame(data)
df.to_csv('links.csv', index=False, encoding='utf-8-sig')

driver.quit()

In [ ]:
# 상세 페이지에서 데이터 가져오는 파트

url = 'https://page.kakao.com'
options = Options()
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36")
data_list = []

# 여기에 로그인 파트

driver = webdriver.Chrome(options=options)

driver.get(url)

# 쿠키 로드
with open('kakao_cookies.pkl', 'rb') as f:
  cookies = pickle.load(f)
  for cookie in cookies:
    driver.add_cookie(cookie)

driver.refresh()

time.sleep(3)

df = pd.read_csv('Links.csv')
for url in df['link'].iloc[20001:40001]:
  data = {}

  try:
    # 두번째 페이지 먼저 호출
    driver.get(url+'?tab_type=about')
    time.sleep(3)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
      
    # JavaScript를 사용하여 버튼이 존재하면 클릭
    script = f"""
    var button = document.querySelector('button.font-medium1.flex.h-59pxr.cursor-pointer.items-center.justify-center.border-0.border-t-1.border-solid.border-line-10.px-24pxr.text-el-50');
    if (button) {{
        button.click();
    }}
    """
    driver.execute_script(script)
    time.sleep(1)

    # 키워드 정보가 없거나, 발행자 정보가 한개 이상이거나, 또는 전자책 정가 정보가 없으면 패스
    keywords = soup.select('div.flex.w-full.flex-wrap.px-18pxr.pb-10pxr > a > div > span')
    if not keywords:
      continue

    others = soup.select('div.flex.w-full.flex-col.items-start.px-18pxr.pb-18pxr > div')[2:]
    if len(others) != 3:
      continue

    check = others[0].select('span.break-all.align-middle')
    if check:
      continue

    title = soup.select_one('span.font-large3-bold.mb-3pxr.text-ellipsis.break-all.text-el-70.line-clamp-2').text
    if '단행본' in title:
      continue
    else:
      data['title'] = title
      
    # 첫번째 페이지 호출
    driver.get(url)

    time.sleep(2)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
      
    for keyword in keywords:
      data['keywords'] = ''.join(keyword.text for keyword in keywords)

    data['publisher'] = others[0].select('span')[1].text
    data['users'] = others[1].select('span')[1].text
    data['price'] = others[2].select('span')[1].text

    author = soup.select_one('span.font-small2.mb-6pxr.text-ellipsis.text-el-70.opacity-70.break-word-anywhere.line-clamp-2').text
    data['author'] = author
    ing = soup.select_one('div.mt-6pxr.flex.items-center > span.font-small2.text-el-70.opacity-70').text
    data['ing'] = ing
    genre = soup.select('span.break-all.align-middle')[1].text
    data['genre'] = genre
    view = soup.select('span.text-el-70.opacity-70')[1].text
    data['viewCount'] = view
    score = soup.select('span.text-el-70.opacity-70')[2].text
    data['score'] = score
    counts = soup.select('span.text-ellipsis.break-all.line-clamp-1.font-small2-bold.text-el-70')
    uploaded = counts[0].text
    data['uploaded'] = uploaded
    review = counts[1].text
    data['reviewCount'] = review
    date = soup.select_one('div.flex.flex-col.pr-14pxr > div > span.break-all.align-middle').text
    data['date'] = date

    free_info = soup.select_one('div.flex.flex-wrap.text-el-70 > span.opacity-70.font-small2').text
    data['free_info'] = free_info
    
    data_list.append(data)
  except:
    continue

df1 = pd.DataFrame(data_list)
df1.to_csv('Data120001-40001.csv', index=False, encoding='utf-8-sig')

driver.quit()

# 네이버 시리즈에서 데이터 크롤링

- 로맨스 페이지 : 1740 / 코드 201
- 로판 페이지 : 561 / 코드 207
- 판타지 페이지 : 466 / 코드 202
- 현판 페이지 : 372 / 코드 208
- 무협 페이지 : 210 / 코드 206

In [8]:
# 네이버 시리즈 카테고리별 링크 추출

base_url = 'https://series.naver.com'
url = 'https://series.naver.com/novel/categoryProductList.series?categoryTypeCode=genre&genreCode=201&orderTypeCode=sale&is&isFinished=false&page={}'

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

data = []
for i in range(1,1741):
  response = requests.get(url.format(i), headers=headers)
  soup = BeautifulSoup(response.text, 'html.parser')

  lst_list = soup.select('ul.lst_list > li')
  for lst in lst_list:
    link = lst.select_one('div > h3 > a')['href']
    data.append(base_url+link)

df = pd.DataFrame({'link':data})
df.to_csv('genre_201.csv', index=False)

In [2]:
import pandas as pd

df = pd.read_csv('SeriesLinks.csv')

In [9]:
url = df['link'][0]

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

synopsis = soup.select('#content > div.end_dsc > div')

print(synopsis)

<br/>환관들의 은밀한 세상에 한 여인이 뛰.. 금녀(禁女)의 구역. 
					
<span class="al_r"><a class="lk_more _toggleMore(synopsis)" href="#" onclick="nclk_v2(event, 'end.storemore', '1200383');">더보기<em></em></a></span>
</div>, <div class="_synopsis" style="display: none">
<br/>그녀의 고민 상담에 구중궁궐이 들썩이기 시작한다.없는 환관까지 되었으니……. 놈이는 사실 남장여인이다. 
						<span class="al_r"><a class="lk_more up _toggleMore(synopsis)" href="#" onclick="nclk_v2(event, 'end.storeclose', '1200383');">접기<em></em></a></span>
</br></br></br></div>]


In [46]:
options = Options()
options.add_argument("--headless")
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36")

driver = webdriver.Chrome(options=options)

datas = []

df = pd.read_csv('SeriesLinks.csv')
count = 0
for link in df['link'].iloc[:10001]:
  count += 1
  driver.get(link)
  time.sleep(1)

  html = driver.page_source

  soup = BeautifulSoup(html, 'html.parser')

  data = {}
  try:
    try:
      synopsis = soup.select_one('div.end_dsc > div._synopsis').get_text().strip()
    except AttributeError:
      synopsis = soup.select('div.end_dsc > div')[1].get_text().strip()
    title = soup.select_one('#content > div.end_head > h2').get_text().strip()
    score = soup.select_one('div.score_area > em').get_text().strip()
    review_count = soup.select_one('span.u_cbox_count').get_text().strip()
    review_list = soup.select('span.u_cbox_contents')
    completed = soup.select_one('li.info_lst > ul > li > span').get_text().strip()
    genre = soup.select_one('li.info_lst > ul > li > span > a').get_text().strip()
    author = soup.select_one('li.info_lst > ul > li > a').get_text().strip()

    data['title'] = title
    data['genre'] = genre
    data['author'] = author
    data['score'] = score
    data['completed'] = completed
    data['review_count'] = review_count
    data['synopsis'] = synopsis
    data['reviews'] = [review.get_text().strip() for review in review_list]

    if count % 100 == 0:
      print(count)
    
  except Exception as e:
    print("Error by", e, link)

    continue

  datas.append(data)

driver.quit()

df1 = pd.DataFrame(datas)
df1.to_csv('SeriesData.csv', index=False, encoding='utf-8-sig')

In [47]:
df2 = pd.read_csv('SeriesData.csv')

df2.head()

,title,genre,author,score,completed,review_count,synopsis,reviews
0,꼬리 아홉 달린 곰 [독점],로맨스,정도령,9.2,완결,"1,322","“세상에 여자는 많죠. 근데 그거 알아요? 나는 한 명이라는 거?” \n조기졸업,...",['오늘 오전에 오늘 업뎃이 없다고 공지를 올렸는데......여기에 공지설정 기능이...
